###  K-means with Euclidena distance as proximity measure

In [258]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import random as rd

In [259]:
def getEucledian(a,b):
    #print(a,b)
    total = 0;
    for i in range(0,len(a)):
        diff = b[i] - a[i];
        total += diff * diff;
    return np.sqrt(total)

In [260]:
def normContinuousAttributes(df,labels):
    for i in labels:
        df[i] = df[[i]].apply(lambda x : (x - np.min(x))/(np.max(x) - np.min(x)))
    return df

In [261]:
def clusterSSE(x,k_clusters,k_centers):
    total = {} 
    for key,value in k_clusters.items():
        s = 0
        center = k_centers[key]
        for i in value:
            s = s + math.pow(getEucledian(x[i],center),2)
        total[key] = s
    return total       

In [262]:
def overallSSE(cluster_sse):
    s = 0
    for key,value in cluster_sse.items():
        s = s + cluster_sse[key]
    return s

In [263]:
def SSB(x,k_clusters,k_centers):
    c = getMeanUsingIndex(x,list(range(0,len(x))))
    keys = list(k_centers.keys())
    s =  0
    for j in keys:
        t = len(k_clusters[j])
        s = s + (t * math.pow(getEucledian(k_centers[j],c),2))
    return s

In [264]:
def getMeanUsingIndex(x,indices):
    total = []
    for i in x[0]:
        total.append(0)
    #print(total)
    for i in indices:
        for j in range(0,len(total)):
            total[j] = total[j] + x[i][j]
    total = [f/len(indices) for f in total]
    return (total)

In [265]:
def anyCenterChanged(k_centers,new_centers):
    for key,value in k_centers.items():
        if(k_centers[key] != new_centers[key]):
            return True
    return False

In [266]:
def assignInputToClusters(x,k_centers):
    #keep track of clusters
    k_clusters = {}
    # print("Centers : ")
    # print(k_centers)
    dissimilarity = []
    #assign all input patterns to clusters
    for key,i in k_centers.items():
        for j in x:
            dissimilarity.append(getEucledian(i,j)) #eucledian similarity

    dissimilarity = np.reshape(dissimilarity,(len(k_centers),len(x)))
    df_dissim = pd.DataFrame(data=dissimilarity)
    #print(df_dissim)
    for i in range(1,df_dissim.shape[0]+1):
        k_clusters[i] = [];
    for i in df_dissim.columns:
        #print(df_dissim.nsmallest(1,i).index[0],end = " ")
        shortest_dist_index = df_dissim.nsmallest(1,i).index[0]
        k_clusters[shortest_dist_index+1].append(i)
    #print(k_clusters)
    new_centers = {}
    for key,value in k_clusters.items():
        #print(key,value)
        new_centers[key]  = getMeanUsingIndex(x,value)
#     print("New Centers : ")
#     print(new_centers)
    if(anyCenterChanged(k_centers,new_centers)):
        assignInputToClusters(x,new_centers)
    return (new_centers,k_clusters)

In [267]:
def recomputeDist(x,centers):
    dist = []
    for i in x:
        t = []
        for key,value in centers.items():
            t.append(getEucledian(i,value))
        dist.append(np.min(t))
    total = np.sum(dist)
    prob_dist = [d/total for d in dist]
    return dist,prob_dist

In [268]:
def kMeansPlusPlus(x,K,first_center):
    k_centers = {}
    #randomly select one center
    #k_centers[1] = rd.sample(x,1)[0]
    k_centers[1] = first_center
    #print("First center : " + str(k_centers[1]))
    #calculate distance of all points to this center
    dist = []
    for i in x:
        dist.append(math.pow(getEucledian(i,k_centers[1]),2))
    total = np.sum(dist)
    #get probability distribution for each point using the distance
    prob_dist = [d/total for d in dist]
    #select k centers using probability distribution
    j = 2
    index = list(range(0,len(x)))
    while(not(len(k_centers) == K)):
        i = np.random.choice(index,p=prob_dist)
        k_centers[j] = x[i]
        dist,prob_dist = recomputeDist(x,k_centers)
        j = j + 1
    return k_centers
    

In [269]:
def kMeans(x,K,kmeans_plus_plus,first_center):
    #select K number of input patterns as cluster centers randomly
    centers = rd.sample(x,K)
    k_centers = {}
    if(kmeans_plus_plus):
        k_centers = kMeansPlusPlus(x,K,first_center)
    else:
        k_centers = {}
        i = 1
        for j in centers:
            k_centers[i] = j
            i = i + 1
    (k_centers,k_clusters) = assignInputToClusters(x,k_centers)
    return (k_clusters,k_centers)


In [270]:
def getOutFrame(k_clusters):
    columns = ["Row ID","Cluster"]
    df_out = pd.DataFrame() 
    for key,values in k_clusters.items():
        t = []
        for i in values:
            t.append(key)
        df = pd.DataFrame({"Row ID" : values, "Cluster" : t})
        df_out = df_out.append(df)
        df_out = df_out.sort_values("Row ID",ascending=True)
#     indices = df_out["Row ID"]
#     indices = [x+1 for x in indices]
#     df_out["Row ID"] = indices
    return df_out

In [271]:
def plotClusters(x,k_clusters,title,xlabel,ylabel):
    fig1 = plt.figure(1)
    for key,value in k_clusters.items():
        c = [x[index] for index in value]
        #print(c)
        plt.scatter(*zip(*c),label="Cluster " + str(key))
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()

In [272]:
def getTrueClusters(df,label):
    k_clusters = {}
    k_centers = {}
    clusters = set(df[label])
    #print(clusters)
    df_temp = df.drop(["ID",label],axis=1)
    x = df_temp.values.tolist()
    for i in clusters:
        a = df.loc[df[label] == i].index
        #print(a)
        k_clusters[i] = list(a)
        k_centers[i] = getMeanUsingIndex(x,list(a))
    #print(k_clusters)
    return k_clusters,k_centers

In [273]:
def crossTabMatrix(k_clusters,true_clusters):
    df = pd.DataFrame()
    t = []
    for key,value in k_clusters.items():
        for a,b in true_clusters.items():
            b3 = [val for val in value if val in b]
            t.append(len(b3))
    t = np.reshape(t,(len(k_clusters),len(true_clusters)))
    columns = list(true_clusters.keys())
    df_temp = pd.DataFrame(data=list(k_clusters.keys()),columns=["Cluster"])
    df = pd.DataFrame(data=t,columns=columns)
    df = pd.concat([df_temp,df],axis=1)
    df = df.set_index('Cluster')
    return df

### Read TwoDimHard.csv

In [274]:
def readTwoDimHard(file):
    df_tdh = pd.read_csv("TwoDimHard.csv")
    true_clusters,true_centers = getTrueClusters(df_tdh,"cluster")
    df_tdh = df_tdh.drop(["ID","cluster"],axis=1)
    x = df_tdh.values.tolist()
    return x,true_clusters,true_centers

### Read Wine.csv

In [275]:
def readWine(file):
    df_wine = pd.read_csv("wine.csv")
    df_wine = df_wine.drop(["class"],axis=1)
    labels = ["fx_acidity","resid_sugar","free_sulf_d","tot_sulf_d","pH","alcohol"]
    df_wine = normContinuousAttributes(df_wine,labels)
    true_clusters,true_centers = getTrueClusters(df_wine,"quality")
    df_wine = df_wine.drop(["ID","quality"],axis=1)
    x = df_wine.values.tolist()
    return x,true_clusters,true_centers

### To run K-means for wine dataset, assign True to wine variable. Otherwise TwoDimHard will be used

In [276]:
wine = False
if(wine):
    x,true_clusters,true_centers = readWine("wine.csv")
else:
    x,true_clusters,true_centers = readTwoDimHard("TwoDimHard.csv")

In [277]:
# wine = True
# df_tdh = pd.read_csv("wine.csv")
# df_tdh = df_tdh.drop(["class"],axis=1)
# #df_tdh['quality'] = df_tdh['quality'].map({3:1,4:2,5:3,6:4,7:5,8:6})
# labels = ["fx_acidity","resid_sugar","free_sulf_d","tot_sulf_d","pH","alcohol"]
# df_tdh = normContinuousAttributes(df_tdh,labels)
# true_clusters,true_centers = getTrueClusters(df_tdh,"quality")
# df_tdh = df_tdh.drop(["ID","quality"],axis=1)
# x = df_tdh.values.tolist()

### True Cluster SSE and SSB

In [278]:
true_sse = clusterSSE(x,true_clusters,true_centers)
print("True Cluster SSE : " + str(true_sse))
true_overall_sse = overallSSE(true_sse)
print("True Overall SSE : " + str(true_overall_sse))
true_ssb = SSB(x,true_clusters,true_centers)
print("True SSB : " + str(true_ssb))

True Cluster SSE : {1: 0.31284771797726363, 2: 0.9025336156215499, 3: 2.4301187182254433, 4: 1.910715466347143}
True Overall SSE : 5.5562155181714
True SSB : 23.748125616409855


### Input k

In [279]:
#first_center = rd.sample(x,1)[0]
k = int(input("Enter K : "))
t = []
t1 = []
k_clusters =  {}
k_centers = {}
best_centers = {}
best_sse = 300
best_ssb = 0
columns = ["Cluster","Overall SSE","SSB"]
#df_table = pd.DataFrame({"Cluster" : [], "Overall SSE" : [], "Overall SSB" : []})
df_table = pd.DataFrame()

Enter K : 4


### n1 = 0 and n2 = 1 : Run K-means once. n1 = 0 and n2 = 10 : Run K-means 10 times

In [280]:
n1 = 0
n2 = 10
first_center = rd.sample(x,1)[0]
for i in range(n1,n2):
    #first_center = rd.sample(x,1)[0]
    print("Iteration : " + str(i))
    (k_clusters,k_centers) = kMeans(x,k,False,first_center)
    #print(k_centers)
    sse = clusterSSE(x,k_clusters,k_centers)
    print("Cluster SSE : " + str(sse))
    overall_sse = overallSSE(sse)
    print("Overall SSE : " + str(overall_sse))
    t.append(overall_sse)
    ssb = SSB(x,k_clusters,k_centers)
    print("SSB : " + str(ssb))
    t1.append(ssb)
    if(overall_sse > best_sse ):
        best_sse = overall_sse
        global best_clusters
        best_clusters = k_clusters
        global best_centers
        best_centers = k_centers
        best_ssb = ssb
        global best_cluster_sse
        best_cluster_sse = sse
    df = pd.DataFrame({"Cluster" : "Assigned Cluster", "Overall SSE" : [overall_sse],"SSB" : [ssb]})
    df_table = df_table.append(df) 
    df = pd.DataFrame({"Cluster" : "True Cluster", "Overall SSE" : [true_overall_sse],"SSB" : [true_ssb]})
    df_table = df_table.append(df)
    df_table = df_table.set_index('Cluster')
df_cross = crossTabMatrix(k_clusters,true_clusters)
df_out = getOutFrame(k_clusters)
if(wine):
    df_out.to_csv("wine_output.csv")
else:
    df_out.to_csv("TwoDimHard_output.csv")

Iteration : 0
Cluster SSE : {1: 1.1917710975363784, 2: 0.26880419553900814, 3: 7.941110013679204, 4: 0.816501698431555}
Overall SSE : 10.218187005186145
SSB : 19.086154129395087
Iteration : 1
Cluster SSE : {1: 3.9594467135778917, 2: 3.0508439785318386, 3: 1.7370539648696388, 4: 0.635489738278328}
Overall SSE : 9.382834395257696
SSB : 19.921506739323537
Iteration : 2
Cluster SSE : {1: 3.1141918257936827, 2: 1.129974384697563, 3: 0.856117709775622, 4: 0.7126898338784103}
Overall SSE : 5.812973754145277
SSB : 23.491367380435975
Iteration : 3
Cluster SSE : {1: 2.309376036185425, 2: 11.683553456451278, 3: 0.06515844653309895, 4: 0.10728478719303286}
Overall SSE : 14.165372726362834
SSB : 15.138968408218442
Iteration : 4
Cluster SSE : {1: 0.5180398743373332, 2: 10.948816277064353, 3: 0.6963904931689064, 4: 0.06932765371535428}
Overall SSE : 12.232574298285947
SSB : 17.0717668362953
Iteration : 5
Cluster SSE : {1: 0.47310201857710343, 2: 5.89776015076315, 3: 0.9260018924891988, 4: 1.655874905

In [281]:
df_cross.head()

,1,2,3,4
Cluster,,,,
1,1,0,88,2
2,0,0,0,47
3,0,0,5,33
4,88,100,4,32


In [282]:
#df_table.head()

In [283]:
best_overall_sse = np.min(t)
print("Overall SSE's : " + str(t))
print("Best SSE : " + str(best_overall_sse))
print("Average SSE : " + str(np.average(t)))
print("Worst SSE : " + str(np.max(t)))
best_overall_ssb = np.min(t1)
print("Overall SSB's : " + str(t1))
print("Best SSB : " + str(best_overall_ssb))
print("Average SSB : " + str(np.average(t1)))
print("Worst SSB : " + str(np.max(t1)))

Overall SSE's : [10.218187005186145, 9.382834395257696, 5.812973754145277, 14.165372726362834, 12.232574298285947, 8.95273896759819, 9.204851149024543, 8.292654642095101, 8.62760674536839, 11.327826365361338]
Best SSE : 5.81297375415
Average SSE : 9.82176200487
Worst SSE : 14.1653727264
Overall SSB's : [19.086154129395087, 19.921506739323537, 23.491367380435975, 15.138968408218442, 17.0717668362953, 20.35160216698304, 20.099489985556662, 21.011686492486138, 20.676734389212854, 17.976514769219868]
Best SSB : 15.1389684082
Average SSB : 19.4825791297
Worst SSB : 23.4913673804


In [284]:
#plt.subplot(1,1,1)
#plotClusters(x,k_clusters,"Clusters with K = " + str(k),"x1","x2")
#plt.subplot(1,1,2)
#plotClusters(x,true_clusters,"True Clusters with K = " + str(len(true_centers)),"x1","x2")
df_out.head()

,Cluster,Row ID
0,4,0
1,4,1
2,4,2
3,4,3
4,4,4


In [285]:
def plot(y,k_list,ylabel,title):
    my_xticks = k_list
    s = list(range(0,len(k_list)))
    plt.xticks(s, my_xticks)
    print(s,y)
    plt.plot(s,y)
    plt.xlabel("k")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

In [286]:
# print(best_sse)
# print(best_cluster_sse)
# print(best_ssb)

### For Wine Dataset

In [287]:
if(wine):
    plot(t,list(range(n1,n2)),"Overall SSE","Overall SSE vs K")
    plot(t1,list(range(n1,n2)),"SSB","SSB vs K")